# Práctica I

## Extracción de los datos

### Funciones de utilidad

Importación de las librerías necesarias

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import requests, json
from IPython.display import IFrame

Se crea la función **`get_raw_data_frame`** que permite extrare el dataframe de PANDAs a partir del código del juego de datos. El segundo de los argumentos es para poder soportar que la respuesta sea devuleta comprimida.

In [2]:
def get_raw_data_frame( key, gziped ): 
    
    # url_template = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/%s$DEFAULTVIEW/?format=TSV&compressed=false' 
    
    url_template =  'https://ec.europa.eu/eurostat/databrowser-backend/api/extraction/1.0/LIVE/false/tsv/%s?i'

    url = url_template % key

    return pd.read_table( url, compression = 'gzip' ) if gziped else  pd.read_table(  url )


Función de flujo de datos: **`extract_country`**

Extraer el país a partir de la primera columna del dataframe

In [3]:
def extract_country( data ):
    
    new_data = data.rename( columns={data.iloc[:, 0].name :'country'} )

    new_data['country'] = new_data['country'].str.replace(r'^.*,(.*)$', r'\1', regex=True)
    
    return new_data


Función de flujo de datos: **`trim_column_names`**

Eliminar espacios en los nombres de las columnas

In [4]:
def trim_column_names( data ):
    
    for col in data.columns:
    
        data = data.rename( columns={col :col.strip()} )
    
    return data
    

Función de flujo de datos: **`clean_numeric_columns`**

Limpia, trata y transforma todas las columnas que son númericas

- Se tratan como vacíos aquellos valores que contienen `:`: No disponible o confidencial
- Se eliminan los indicadores de valor estimado o que su definición difieren: Estos valores se tratarán como reales
- Se eliminan los espacios sobrantes

In [5]:
def clean_numeric_columns( data ):
   
   data.iloc[:,1:] = data.iloc[:,1:].replace(
        r'^.*[:].*$', None, regex=True # Not available and confidencial flag
   ).replace(
        r'e', '', regex=True # Remove flag estimated
   ).replace(
        r'd', '', regex=True # Remove flag definition differs
   ).replace( 
        r'^(.+) +$', r'\1', regex = True # rTrim
   ).replace( 
        r'^ +(.+)$', r'\1', regex = True # lTrim
   )
   
   for col in  data.iloc[:,1:].columns :    
        data[col] = pd.to_numeric( data[col] )
    
   return data
    
    

Función de flujo de datos: **`filter_data`**

Función que filtra los registros que cumplann la condión del filtro en la primera de las columnas

In [6]:
def filter_data( filter ): 
    def _filter_data( data ):
        new_data = data[data.iloc[:, 0].str.contains( filter )]    
        new_data.reset_index(inplace = True, drop = True)
        return new_data
    return _filter_data
    

Función que obtiene el maestro de países (**`get_country_names`**), que permite obtener el nombre del país a partir de su identificador

In [7]:
def get_country_names():
    url = requests.get("https://ec.europa.eu/eurostat/databrowser-backend/api/codelist/LIVE/GEO/getCodeListJson/9.0/ESTAT/en/false")
    text = url.text
    data = json.loads(text)
    return data['category']['label']


Función de flujo de datos: **`add_column_country_name_flow`**

Función que añade la columna con el nombre de los países

In [8]:
def add_column_country_name_flow():    
    
    def _add_column_country_name(data) : 
        country_names = get_country_names()    
        data.insert(
            1, 
            'country_name', 
            data.country.map(lambda v: country_names[v] ), 
            True
        )
        return data
    
    return _add_column_country_name 

Función **`export_dataframe`** que permite exportar el dataframe a fichero CSV

In [9]:
def export_dataframe( df, file_name, directory ):
    #file = 'C:\Python_Projects\TipologiaYCicloDeVidaDelDato\Data\%s/%s.csv' % (directory, file_name )
    file = '/home/jovyan/work/%s/%s.csv' % (directory, file_name )
    df.to_csv(file )
    
    

#### Flujo de procesamiento de datos de los dataset auxiliares

Función que realiza todo el proceso de extración y limpieza de los datos.

La función **`flow`** define, por composición de funciones, el flujo procesamiento que se va a realizar después de la carga de cada subdataframe después de su obtención a partir de la petición HTTP:

1) Filtrará los datos por el filtro
2) Obtendrá la columna de los identificadores de los paises a partir de la informaciónb de la primera columna
3) Eliminará los espacios de los nombres de las columnas de los dataframe
4) Procesará todas aquellas columnas relativas a los años para que contengan datos númericos
5) Se añade la columna con la descripción del país

La función **`dataframe_by_key`** combina la obtención del dataframe con su procesamiento posterior

In [10]:
class Compose:
    _f = None
    def __init__(self, f):
        self._f = f
    def andThen( self, g ):
        return Compose( lambda s: ( g( self._f(s) ) ) )
    def apply(self, a): 
        return self._f( a )
    
column_country_name = add_column_country_name_flow()    

def flow( filter ) :
    return Compose( 
        filter_data( filter )
    ).andThen(
        extract_country
    ).andThen(
        trim_column_names 
    ).andThen( 
        clean_numeric_columns 
    ).andThen( 
        column_country_name
    )

def dataframe_by_key( key, filter, gziped = False ):
        return flow(filter).apply(  get_raw_data_frame( key, gziped ) )


### _DATASET I_: Precio de Gas doméstico en € por kw/h

Obtenido del origen de datos [Gas prices components for household consumers - annual data](https://ec.europa.eu/eurostat/databrowser/view/nrg_pc_202_c/default/table?lang=en)

Clave de identificación de los datos: **`NRG_PC_202_C`**

Se filtrarán los datos por:

 - Datos anuales
 - Componentes del precio de la energia: _"Energia y suministro "_
 - Consumición de la energía: En Giga Julios en todas las bandas
 - Moneda: Euro (€)
 - Unidad de medida: kWh

In [11]:
data_gas_prices_household_consumers = dataframe_by_key( 
    key    = 'NRG_PC_202_C', 
    filter = 'A,NRG_SUP,TOT_GJ,EUR,KWH'
) 

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

Columnas del dataset:

In [ ]:
display( data_gas_prices_household_consumers.dtypes )

Se muestran los 10 primeros valores:

In [ ]:
data_gas_prices_household_consumers.head( 10 )

Exportación del dataframe en el directorio `subdataset`

In [ ]:
export_dataframe( 
    df = data_gas_prices_household_consumers, 
    file_name = 'data_gas_prices_household_consumers', 
    directory = 'subdataset' 
) 

### _DATASET II_: Precio de Gas no doméstico en € por kw/h

Obtenido del origen de datos [Gas prices components for non-household consumers - annual data](https://ec.europa.eu/eurostat/databrowser/view/nrg_pc_203_c/default/table?lang=en)

Clave de identificación de los datos: **`NRG_PC_203_C`**

Se filtrarán los datos por:

 - Datos anuales
 - Componentes del precio de la energia: _"Energia y suministro "_
 - Consumición de la energía: En Giga Julios en todas las bandas
 - Moneda: Euro (€)
 - Unidad de medida: kWh

In [ ]:
data_gas_prices_no_household_consumers = dataframe_by_key( 
    key    = 'NRG_PC_203_C', 
    filter = 'A,NRG_SUP,TOT_GJ,EUR,KWH'
) 

Columnas del dataset:

In [ ]:
display( data_gas_prices_no_household_consumers.dtypes )

Se muestran los 10 primeros valores:

In [ ]:
data_gas_prices_no_household_consumers.head( 10 )

Exportación del dataframe en el directorio `subdataset`

In [ ]:
export_dataframe( 
    df = data_gas_prices_no_household_consumers, 
    file_name = 'data_gas_prices_no_household_consumers', 
    directory = 'subdataset' 
) 

### _DATASET III_: Precio de la electricidad doméstica para la franja de 2.500 a 4.999 kWh

Obtenido del origen de datos [Electricity prices components for household consumers - annual data (from 2007 onwards)](https://ec.europa.eu/eurostat/databrowser/view/NRG_PC_204_C__custom_2388428/default/table?lang=en)

Clave de identificación de los datos: **`NRG_PC_204_C__custom_2388428`**

Se filtrarán los datos por:

 - Datos anuales
 - Consumición de la energía: Consumición entre 2500 kWh y 4999 kWh
 - Componentes del precio de la energia: _"Energia y suministro "_
 - Moneda: Euro (€)
 - Unidad de medida: €/kWh

In [ ]:
data_electricity_prices_household_consumers = dataframe_by_key( 
    key    = 'NRG_PC_204_C__custom_2388428', 
    filter = 'A,KWH2500-4999,NRG_SUP,EUR', 
    gziped = True 
) 

Columnas del dataset:

In [ ]:
display( data_electricity_prices_household_consumers.dtypes )

Se muestran los 10 primeros valores:

In [ ]:
data_electricity_prices_household_consumers.head(10)

Exportación del dataframe en el directorio `subdataset`

In [ ]:
export_dataframe( 
    df = data_electricity_prices_household_consumers, 
    file_name = 'data_electricity_prices_household_consumers', 
    directory = 'subdataset' 
) 

### _DATASET IV_: Precio de la electricidad no doméstica

Obtenido del origen de datos [Electricity prices components for non-household consumers - annual data (from 2007 onwards)](https://ec.europa.eu/eurostat/databrowser/view/nrg_pc_205_c/default/table?lang=en)

Clave de identificación de los datos: **`NRG_PC_205_C`**

Se filtrarán los datos por:

 - Datos anuales
 - Componentes del precio de la energia: _"Energia y suministro "_
 - Consumición de la energía: Consumición menos de 20 MWh 
 - Moneda: Euro (€)
 - Unidad de medida: €/kWh

In [ ]:
data_electricity_prices_no_household_consumers = dataframe_by_key(
    key    = 'NRG_PC_205_C', 
    filter = 'A,NRG_SUP,MWH_LT20,EUR'
) 

Columnas del dataset:

In [ ]:
display( data_electricity_prices_no_household_consumers.dtypes )

Se muestran los 10 primeros valores:

In [ ]:
data_electricity_prices_no_household_consumers.head( 10 )

Exportación del dataframe en el directorio `subdataset`

In [ ]:
export_dataframe( 
    df = data_electricity_prices_no_household_consumers, 
    file_name = 'data_electricity_prices_no_household_consumers', 
    directory = 'subdataset' 
) 

## Análisis inicial y procesamiento de los datos

### Funciones de utilidad

Estimador utilizando la media **`median_stimator`**. Dentro de una columna del dataframe evalúa aquellos valores nulos y lo sustituye por la media de los valores que no lo son

In [ ]:
def median_stimator( df, column ) :    
    median = df.loc[pd.notnull( df[column]), column].median()
    df[column].fillna(median,inplace=True)    
    return df    


Función **`show_boxplot`** que muestra el diagrama de caja de los valores de un dataframe


In [ ]:
def show_boxplot( df ):
    sns.set_theme( style = "whitegrid" )
    ax = sns.boxplot( data = df )
    

Función **`init_outlier`**. Iializa el Outlier de una columna, inicializa a nulo el valor máximo de la columna

In [ ]:
def init_outlier(df, column): 
    df.loc[
        df[column] == df[column].max(),
        column 
    ] = np.nan
    return df


### Datos de los costes del gas doméstico


Sustituimos NaN values por su media

In [ ]:
GasPricesHousehold=data_gas_prices_household_consumers

GasPricesHousehold = median_stimator( GasPricesHousehold, '2021' )
GasPricesHousehold = median_stimator( GasPricesHousehold, '2020' )
GasPricesHousehold = median_stimator( GasPricesHousehold, '2019' )
GasPricesHousehold = median_stimator( GasPricesHousehold, '2018' )
GasPricesHousehold = median_stimator( GasPricesHousehold, '2017' )


Se muestran los 10 primeros valores:

In [ ]:
display( GasPricesHousehold.head(10) )

Diagrama de caja para los diferentes años:

In [ ]:
show_boxplot( GasPricesHousehold )

### Datos de los costes del gas para empresas


Sustituimos NaN values por su media

In [ ]:
GasPricesNoHousehold = data_gas_prices_no_household_consumers

GasPricesNoHousehold = median_stimator( GasPricesNoHousehold, '2021' )
GasPricesNoHousehold = median_stimator( GasPricesNoHousehold, '2020' )
GasPricesNoHousehold = median_stimator( GasPricesNoHousehold, '2019' )
GasPricesNoHousehold = median_stimator( GasPricesNoHousehold, '2018' )
GasPricesNoHousehold = median_stimator( GasPricesNoHousehold, '2017' )

Se muestran los 10 primero valores:

In [ ]:
display( GasPricesNoHousehold.head(10) )

Diagrama de caja para los diferentes años:

In [ ]:
show_boxplot( GasPricesNoHousehold )

### Datos de los costes de la electricidad doméstica


Se eliminan las columnas correspondientes a los valores semestrales de lo años desde el 2012 al 2016 que no contienen datos

In [ ]:
ElectPricesHouseholds = data_electricity_prices_household_consumers.drop(
    [
     '2012-S2',
     '2013-S2',
     '2014-S2',
     '2015-S2',
     '2016-S2'
    ], axis=1
)

Muestra de los 10 primeros valores

In [ ]:
display( ElectPricesHouseholds.head( 10 ) )

Se muestra el diagrama de caja

In [ ]:
show_boxplot( ElectPricesHouseholds )

Detectamos Outlier en el año 2021, inicializamos valor

In [ ]:
ElectPricesHouseholds = init_outlier(ElectPricesHouseholds, '2021')

 Estimamos valores nulos por la media

In [ ]:
ElectPricesHouseholds = median_stimator( ElectPricesHouseholds, '2021' )
ElectPricesHouseholds = median_stimator( ElectPricesHouseholds, '2020' )
ElectPricesHouseholds = median_stimator( ElectPricesHouseholds, '2019' )
ElectPricesHouseholds = median_stimator( ElectPricesHouseholds, '2018' )
ElectPricesHouseholds = median_stimator( ElectPricesHouseholds, '2017' )


Se vuelve a mostrar el diagrama de caja

In [ ]:
show_boxplot( ElectPricesHouseholds )

### Datos de los costes de la electricidad para empresas


Se eliminan las columnas correspondientes a los valores semestrales de lo años desde el 2012 al 2016 que no contienen datos

In [ ]:
ElectPricesNoHouseholds = data_electricity_prices_no_household_consumers.drop(
    [
        '2007-S2',
        '2008-S2',
        '2009-S2',
        '2010-S2',
        '2011-S2',
        '2012-S2',
        '2013-S2',
        '2014-S2',
        '2015-S2',
        '2016-S2'
    ], axis=1
)

Se muestra el diagrama de caja

In [ ]:
show_boxplot( ElectPricesNoHouseholds )

Muestra de los 10 primeros valores

In [ ]:
display( ElectPricesNoHouseholds.head( 10 ) )

Detectamos Outlier en los valores del 2021 y lo inicializamos 

In [ ]:
ElectPricesHouseholds = init_outlier(ElectPricesNoHouseholds, '2021')

Se evaluan con el estimador de la media los valores nulos

In [ ]:
ElectPricesNoHouseholds = median_stimator( ElectPricesNoHouseholds, '2021' )
ElectPricesNoHouseholds = median_stimator( ElectPricesNoHouseholds, '2020' )
ElectPricesNoHouseholds = median_stimator( ElectPricesNoHouseholds, '2019' )
ElectPricesNoHouseholds = median_stimator( ElectPricesNoHouseholds, '2018' )
ElectPricesNoHouseholds = median_stimator( ElectPricesNoHouseholds, '2017' )


Se vuelve a mostrar el diagrama de caja

In [ ]:
show_boxplot( ElectPricesNoHouseholds )

### _Conclusión de análisis inicial de datos:_

Descartamos continuar el análisis del "Gas" en el caso de los precios de los consumos de las empresas, ya que los datos obtenidos son una muestra demasiado pequeña, en concreto en los años 2017 y 2018 (ver dataset [data_gas_prices_no_household_consumers.csv](subdataset/data_gas_prices_no_household_consumers.csv)).

# Generación del dataset Final de trabajo

Generamos El Dataset final a exportar, con los datos posibles 

In [ ]:
dElectCol = pd.merge(
    ElectPricesHouseholds.drop(['country_name'], axis=1), 
    ElectPricesNoHouseholds.drop(['country_name'], axis=1), 
    on='country',
    suffixes=('_ElectHouse', '_ElectNoHouse')
)

dGasCol = pd.merge(
    GasPricesHousehold.drop(['country_name'], axis=1), 
    GasPricesNoHousehold.drop(['country_name'], axis=1), 
    on='country',
    suffixes=('_GasHouse', '_GasNoHouse')
)

dEnergyCol = pd.merge(
    dElectCol, 
    dGasCol, 
    on='country'
)



Mostramos el dataset final a publicar, que no será el mismo que utilizemos para analizar

In [ ]:
dEnergyCol

Construimos un Dataset Final para proceder a analizar con los precio de Gas y Electricidad desde 2017 hasta 2021.

In [ ]:
dEnergyHouseCol = pd.merge(
    ElectPricesHouseholds.drop(['country_name'], axis=1), 
    GasPricesHousehold.drop(['country_name'], axis=1), 
    on='country',
    suffixes=('_ElectHouse', '_GasHouse')
)


Se añade la columna del nombre del país

In [ ]:
dEnergyHouseCol= column_country_name( dEnergyHouseCol )
dEnergyCol= column_country_name( dEnergyCol )

Columnas del dataset a Publicar:

In [ ]:
display( dEnergyCol.dtypes )

Columnas del dataset a Analizar:

In [ ]:
display( dEnergyHouseCol.dtypes )

## Exportación dataset Final en formato CSV

In [ ]:
export_dataframe( 
    df = dEnergyCol, 
    file_name = 'energy_price_dataset', 
    directory = 'dataset' 
) 

## Reportes

### Preparación y generación de datasets para Analizar

##### Eliminamos el valor "EU27_2020"

In [ ]:
dEnergyHouseTemp = dEnergyHouseCol.loc[(dEnergyHouseCol["country"] != "EU27_2020")]

##### Trasponemos y recreamos el dataset para prepararlo para una nueva presentación multivariable

In [ ]:
dEnergyHouseDummy_2017=  dEnergyHouseTemp.loc[:,['country', '2017_ElectHouse']]
dEnergyHouseDummy_2017['Year']=2017
dEnergyHouseDummy_2017['Type']='Electricity'
dEnergyHouseDummy_2018=  dEnergyHouseTemp.loc[:,['country', '2018_ElectHouse']]
dEnergyHouseDummy_2018['Year']=2018
dEnergyHouseDummy_2018['Type']='Electricity'
dEnergyHouseDummy_2019=  dEnergyHouseTemp.loc[:,['country', '2019_ElectHouse']]
dEnergyHouseDummy_2019['Year']=2019
dEnergyHouseDummy_2019['Type']='Electricity'
dEnergyHouseDummy_2020=  dEnergyHouseTemp.loc[:,['country', '2020_ElectHouse']]
dEnergyHouseDummy_2020['Year']=2020
dEnergyHouseDummy_2020['Type']='Electricity'
dEnergyHouseDummy_2021= dEnergyHouseTemp.loc[:,['country', '2021_ElectHouse']]
dEnergyHouseDummy_2021['Year']=2021
dEnergyHouseDummy_2021['Type']='Electricity'

dGasHouseDummy_2017=  dEnergyHouseTemp.loc[:,['country', '2017_GasHouse']]
dGasHouseDummy_2017['Year']=2017
dGasHouseDummy_2017['Type']='Gas'
dGasHouseDummy_2018=  dEnergyHouseTemp.loc[:,['country', '2018_GasHouse']]
dGasHouseDummy_2018['Year']=2018
dGasHouseDummy_2018['Type']='Gas'
dGasHouseDummy_2019=  dEnergyHouseTemp.loc[:,['country', '2019_GasHouse']]
dGasHouseDummy_2019['Year']=2019
dGasHouseDummy_2019['Type']='Gas'
dGasHouseDummy_2020=  dEnergyHouseTemp.loc[:,['country', '2020_GasHouse']]
dGasHouseDummy_2020['Year']=2020
dGasHouseDummy_2020['Type']='Gas'
dGasHouseDummy_2021= dEnergyHouseTemp.loc[:,['country', '2021_GasHouse']]
dGasHouseDummy_2021['Year']=2021
dGasHouseDummy_2021['Type']='Gas'


In [ ]:
dEnergyHouseDummy_2017=dEnergyHouseDummy_2017.rename(columns={"2017_ElectHouse": "Price"})
dEnergyHouseDummy_2018=dEnergyHouseDummy_2018.rename(columns={"2018_ElectHouse": "Price"})
dEnergyHouseDummy_2019=dEnergyHouseDummy_2019.rename(columns={"2019_ElectHouse": "Price"})
dEnergyHouseDummy_2020=dEnergyHouseDummy_2020.rename(columns={"2020_ElectHouse": "Price"})
dEnergyHouseDummy_2021=dEnergyHouseDummy_2021.rename(columns={"2021_ElectHouse": "Price"})
dGasHouseDummy_2017=dGasHouseDummy_2017.rename(columns={"2017_GasHouse": "Price"})
dGasHouseDummy_2018=dGasHouseDummy_2018.rename(columns={"2018_GasHouse": "Price"})
dGasHouseDummy_2019=dGasHouseDummy_2019.rename(columns={"2019_GasHouse": "Price"})
dGasHouseDummy_2020=dGasHouseDummy_2020.rename(columns={"2020_GasHouse": "Price"})
dGasHouseDummy_2021=dGasHouseDummy_2021.rename(columns={"2021_GasHouse": "Price"})


dEnergyHouse = pd.concat(
        [
            dEnergyHouseDummy_2017,
            dEnergyHouseDummy_2018,
            dEnergyHouseDummy_2019,
            dEnergyHouseDummy_2020,
            dEnergyHouseDummy_2021,
            dGasHouseDummy_2017,
            dGasHouseDummy_2018,
            dGasHouseDummy_2019,
            dGasHouseDummy_2020,
            dGasHouseDummy_2021
        ]

    )

display(dEnergyHouse)

### Análisis Visual de la información

#### Total de Coste de Tipo de Energía por paises en los 4 últimos años

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
sns.barplot(x='country', y='Price', hue='Type', data=dEnergyHouse, saturation=0.8)


#### Conclusiones de Análisis Visual

Teniendo en cuenta la información contenida en el siguiente Link del Banco Mundial, donde podemos obtner el PIB de cualquier país

In [ ]:
IFrame(src="https://datos.bancomundial.org/share/widget?indicators=NY.GDP.PCAP.CD&view=map", width='450', height='300', frameBorder='0', scrolling="no")

https://datos.bancomundial.org/share/widget?indicators=NY.GDP.PCAP.CD&view=map

Obtenemos conclusiones como que por Ejemplo, España siendo uno de los paises con peor PIB de Europa, tiene uno de los costes acumulados históricos de los últimos 4 años más altos de Europa. 
O que Países con PIB más alto de la Comunidad Europea como Alemania (GE) o Austria (AT) tienen de los precios más bajos del continente Europes.
Así mismo se observa una varianza reseñable en los precios de la Electricidad por cada país, que no se observa de una forma equivalente en el Gas.

#### Detalle histórico de los 4 últimos años de coste por tipo de Energía y país

In [ ]:
dEnergyHouseElect=dEnergyHouse[(dEnergyHouse['Type'] == 'Electricidad') ]
dEnergyHouseGas=dEnergyHouse[(dEnergyHouse['Type'] == 'Gas') ]


sns.set(font_scale = 1.60)
g=sns.relplot(data=dEnergyHouse, x='country', y='Price', size='Type', hue="Year")
g.fig.set_size_inches(45,15)

#### Conclusiones de Análisis Visual

Alemania mantiene constante practicamente su coste de Gas igual en los últimos 4 años, España marca máximo histórico en Gas y su segundo máximo en Electricidad, teniendo una varianza reseñable anual en los precios. Reino Unido tiene el precio más bajo en Electricidad y Gas de los últimos 4 años.